# 12 - Exception Handling and Recovery 
- To illustrate Exception Handling and Recovery, we can demonstrate how an agent might handle a tool call that fails, either due to an invalid input or an external service error.


In [ ]:
from google.adk.agents import Agent


# Conceptual Tools
def get_precise_location_info(address: str) -> dict:
    """Primary tool: gets precise geolocation. Might fail."""
    if "problematic" in address.lower():
        return {
            "status": "error",
            "error_message": "Precise location service unavailable for this address.",
        }
    return {
        "status": "success",
        "latitude": 40.7128,
        "longitude": -74.0060,
        "precision": "high",
    }


def get_general_area_info(city: str) -> dict:
    """Fallback tool: gets general information if precise fails."""
    return {
        "status": "success",
        "area_info": f"General information for {city}: It's a known city.",
    }


location_agent_with_fallback = Agent(
    name="location_agent_fallback",
    model="gemini-2.0-flash-exp",
    description="Tries to get precise location, falls back to general info on failure.",
    instruction="""
You need to provide location information.
1. First, try to use the get_precise_location_info tool with the full address.
2. If get_precise_location_info returns "status": "error", then try to extract the city from the address.
3. Use the get_general_area_info tool with just the city as a fallback.
4. If both fail, or if you cannot extract a city, apologize that location info could not be retrieved.
Present information clearly from whichever tool succeeds.
    """,
    tools=[get_precise_location_info, get_general_area_info],
)

# --- Conceptual Run ---
# User: "Tell me about problematic address, Anytown"
# Agent calls: get_precise_location_info(address="problematic address, Anytown")
# Tool returns: {"status": "error", "error_message": "Precise location service unavailable for this address."}
# Agent (following instructions) extracts "Anytown", then calls: get_general_area_info(city="Anytown")
# Tool returns: {"status": "success", "area_info": "General information for Anytown: It's a known city."}
# Agent responds: "I couldn't get precise data, but here's general information for Anytown: It's a known city."